In [1]:
import datetime
import csv
import os

In [2]:
import random
import numpy as np

In [3]:
import pandas as pd


In [4]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot

In [5]:
path = 'data/states.csv'
DF = pd.read_csv(path)
DF['date'] = pd.to_datetime(DF['date'])
DF['date'] = [datetime.datetime(x.year, x.month, x.day) for x in DF['date']]
DF.head()

,date,state,cases,deaths
0,2020-01-22,Washington,0,0
1,2020-01-23,Washington,0,0
2,2020-01-24,Washington,0,0
3,2020-01-25,Illinois,0,0
4,2020-01-25,Washington,0,0


In [6]:
import scipy.stats
from scipy.stats import probplot
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
#texas opened up April 30
df_texas = DF[(DF['date'] > datetime.datetime(2020, 4, 30)) 
            & (DF['state'] == 'Texas')]
df_weeks = list(range(len(df_texas['date'])))
df_texas['week'] = df_weeks


/home/henry/Envs2/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
x = df_texas['week']
y = df_texas['deaths']
p = figure(title = 'Texas')
p.circle(x = x, y = y)
model = smf.ols(formula='deaths ~ week' , data=df_texas)
a, b1 = res.params
y_fit = [a + b1 *x for x in x]
res = model.fit()
print(res.pvalues[1])
p.line(x = x, y = y_fit)
show(p)

NameError: name 'res' is not defined

In [ ]:
x = df_texas['week']
y = df_texas['cases']
p = figure(title = 'Texas')
p.circle(x = x, y = y)
model = smf.ols(formula='cases ~ week' , data=df_texas)
a, b1 = res.params
y_fit = [a + b1 * x for x in x]
res = model.fit()
print(res.pvalues[1])
p.line(x = x, y = y_fit)
show(p)

In [13]:
def make_linear(df, state, key, date, title = None, plot_height = 250,
               plot_width = 250):
    df = DF[(DF['date'] >= date) 
            & (DF['state'] == state)]
    day = list(range(len(df['date'])))
    df['day'] = day
    x = df['day']
    y = df[key]
    p = figure(title = title, plot_width = plot_width, plot_height = plot_height)
    p.circle(x = x, y = y)
    model = smf.ols(formula='{k} ~ day'.format(k = key) , data=df)
    res = model.fit()

    a, b1 = res.params
    y_fit = [a + b1 * x for x in x]
    #print(res.pvalues[1])
    p.line(x = x, y = y_fit)
    return p
p_texas_cases = make_linear(df = DF, state= 'Texas', key = 'cases', 
            date = datetime.datetime(2020, 4, 30), title= 'Texas Cases')

p_arkansas_cases = make_linear(df = DF, state= 'Arkansas', key = 'cases', 
            date = datetime.datetime(2020, 4, 22), title= 'Arkansas Cases')
grid = gridplot([p_texas_cases,p_arkansas_cases], ncols = 4)
show(grid)


/home/henry/Envs2/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
def all_states(df):
    graphs = []
    for state in sorted(set(df['state'].tolist())):
        p = make_linear(df = DF, state= state, key = 'cases', 
            date = datetime.datetime(2020, 4, 30), title= '{s} Cases'.format(s = state))
        graphs.append(p)
    return graphs
g = all_states(DF)
grid = gridplot(g, ncols = 4)
show(grid)


/home/henry/Envs2/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
